In [2]:
# 간단한 예제 데이터셋 생성

import pandas as pd

df = pd.DataFrame([
    ['green', 'M', 10.1, 'class1'],
    ['red', 'L', 13.5, 'class2'],
    ['blue', 'XL', 15.3, 'class1'],
])

# color: 순서가 없는 특성
# size: 순서가 있는 특성
# price: 수치형 특성
# classlabel: 순서가 없는 클래스 레이블(순서가 있는 클래스 레이블은 사이킷런이 지원하지 않음)
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [3]:
# 문자열 값을 정수로 바꾸기

size_mapping = { 'XL': 3, 'L': 2, 'M': 1 }
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [4]:
# 변화된 정수값을 다시 원래의 문자열로 바꾸기 

inv_size_mapping = { v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

In [5]:
# 클래스 레이블을 정수값으로 인코딩하기 

import numpy as np

class_mapping = { label:idx for idx,label in enumerate(np.unique(df['classlabel'])) }
class_mapping

{'class1': 0, 'class2': 1}

In [6]:
# 정수로 변환된 클래스 레이블을 테스트 데이터에 반영하기

df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [7]:
# 변환된 클래스를 다시 원본 문자열로 바꾸기

inv_class_mapping = { v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [8]:
# 사이킷런의 LabelEncoder를 사용해서 클래스 레이블 인코딩하기

from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

In [9]:
# 사이킷런의 LabelEncoder를 사용해서 클래스 레이블 디코딩하기

class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

In [10]:
# 순서가 없은 color 특성을 정수로 인코딩하기
# blue = 0, green = 1, red = 2

x = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
x[:, 0] = color_le.fit_transform(x[:, 0])
x

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

In [11]:
# color는 순서가 없는 특성인데 정수값으로 인코딩되어 순서가 있는 값이 되었다.
# 이런 문제를 해결하는 방법이 원-핫 인코딩이다.
# 원-핫 인코딩은 color 특성을 세개의 새로운 특성 blue, green, red로 변환한다.(이진값 사용)

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# auto: 경고 메시지를 피하고 고유한 값(정수 또는 문자열)을 사용하는 방식
oh_enc = OneHotEncoder(categories='auto')
col_trans = ColumnTransformer([('oh_enc', oh_enc, [0])], remainder='passthrough')
col_trans.fit_transform(x)

array([[0.0, 1.0, 0.0, 1, 10.1],
       [0.0, 0.0, 1.0, 2, 13.5],
       [1.0, 0.0, 0.0, 3, 15.3]], dtype=object)

In [12]:
# 판다스의 get_dummies 함수를 사용한 원-핫 인코딩

pd.get_dummies(df[['price', 'color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


원-핫 인코딩된 데이터셋을 사용할때는 다중 공선성(multicollinearity)에 유념해야 한다.<br/>
예를들어 역행렬을 구해야하는 알고리즘 등에는 문제가 될 수 있다.<br/>
특성 간 상관관계가 높으면 역행렬을 계산하기 어려워서 수치적으로 불안정해지기 때문이다.<br/>
특성 간 상관관계를 감소시키려면 원-핫 인코딩된 배열에서 특성 열 하나를 삭제한다.<br/>
이렇게 특성을 삭제해도 잃는 정보는 없다.<br/>
예를들어 color_blue 열을 삭제해도 color_green=0이고 color_red=0이면 blue라는 것을 알 수 있다.<br/>

In [13]:
# 특성 상관관계 감소를 위해서 drop_first로 첫번째 열 삭제하기

pd.get_dummies(df[['price', 'color', 'size']], drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


In [15]:
# OneHotEncoder로 첫번째 열 삭제하기

oh_enc = OneHotEncoder(categories='auto')
col_trans = ColumnTransformer([('oh_enc', oh_enc, [0])], remainder='passthrough')
col_trans.fit_transform(x)[:, 1:]



array([[1.0, 0.0, 1, 10.1],
       [0.0, 1.0, 2, 13.5],
       [0.0, 0.0, 3, 15.3]], dtype=object)